In [14]:
!gdown 1sCTQXp9kTcm8_jckqC-v_gWxIsFCcUbS
!gdown 1vK24A09o5Nev5qj1qNhndFe6beTWSDRU
!gdown 1THvOuf_EOn6c_6TLy0Bqs23BP2NraBR2

Downloading...
From: https://drive.google.com/uc?id=1sCTQXp9kTcm8_jckqC-v_gWxIsFCcUbS
To: /content/X_train
100% 7.14M/7.14M [00:00<00:00, 95.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vK24A09o5Nev5qj1qNhndFe6beTWSDRU
To: /content/Y_train
100% 65.1k/65.1k [00:00<00:00, 45.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1THvOuf_EOn6c_6TLy0Bqs23BP2NraBR2
To: /content/X_test
100% 3.57M/3.57M [00:00<00:00, 161MB/s]


In [15]:
import numpy as np
import pandas as pd
import csv
from sklearn.svm import SVC

In [16]:
X = pd.read_csv("X_train")  # 'age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss' are coninuous, others are discrete
y = pd.read_csv("Y_train", header = None).values.reshape(-1)

In [17]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
seed = 666
np.random.seed(seed)

In [18]:
selector = SelectKBest(chi2, k=60)
selector.fit(X, y)
cols = selector.get_support(indices=True)
X_new = X.iloc[:,cols]
KEYS = X_new.keys()
KEYS=KEYS.tolist()
print(KEYS)
dele = [' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th',' 10th',' 11th',' 12th', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college' \
        ,' Divorced', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed']
for word in dele: 
    KEYS.remove(word)
KEYS.append('edu_degree')
KEYS.append('marriage')

['age', 'fnlwgt', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', ' Federal-gov', ' Local-gov', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', '?_workclass', ' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college', ' Divorced', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed', ' Adm-clerical', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving', '?_occupation', ' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife', ' Amer-Indian-Eskimo', ' Black', ' Other', ' White', ' Columbia', ' Dominican-Republic', ' El-Salvador', ' India', ' Mexico']


In [19]:
def selection(X,keys):
    selX = pd.DataFrame.from_dict({k: X[k] for k in keys})
    return selX
def normalize(X,flag=False, mu_x=None, std_x=None):
  #pass
    if flag==False:
        mu_x= []
        std_x =[]
    else:
        count=0
    for key in ['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss',"edu_degree"]:
        X[key]=(X[key]-X[key].mean())/X[key].std()
#         if  flag==False:
#             mu_x.append(X[key].mean())
#             std_x.append(X[key].std())
#             X[key]=(X[key]-X[key].mean())/X[key].std()
#         else:
#             X[key]=(X[key]-mu_x[count])/std_x[count]
            
        #count+=1
    return X, mu_x, std_x
def education_redesign(X):
    X["edu_degree"]= X[" HS-grad"]*1+ X[" Bachelors"]*2+ X[" Some-college"]*2+ X[" Assoc-acdm"]*3+ X[" Assoc-voc"]*3 \
    + X[" Masters"]*4+ X[" Prof-school"]*5+ X[" Doctorate"]*5 
    return X
def marriage_redesign(X):
    X["marriage"]= X[' Married-civ-spouse']*1+ X[' Married-spouse-absent']*1+ X[' Married-AF-spouse']*1+ X[" Divorced"]*0+ X[" Separated"]*0 \
    + X[" Widowed"]*0+ X[" Never-married"]*-1 
    return X
def load_train(mu,std):
    X = pd.read_csv("X_train")  # 'age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss' are coninuous, others are discrete
    Y = pd.read_csv("Y_train", header = None).values.reshape(-1)
    X = education_redesign(X)
    X = marriage_redesign(X)
    X,non,non = normalize(X,True,mu,std)
    print(len(X))
    X = selection(X,KEYS)
    return X, Y

def load_test():
    X = pd.read_csv("X_test")
    print(len(X))
    X = education_redesign(X)
    X = marriage_redesign(X)
    X,mu,std = normalize(X,False)
    X = selection(X,KEYS)
    print(len(X))
    return X,mu,std

In [20]:
test_X ,mu,std= load_test()
pd.set_option('display.max_rows', None)
train_X, train_Y= load_train(mu,std)
print(train_X.shape)
print(test_X.shape)
for key in train_X.keys():
    print(key)


16281
16281
32561
(32561, 42)
(16281, 42)
age
fnlwgt
sex
capital_gain
capital_loss
hours_per_week
 Federal-gov
 Local-gov
 Private
 Self-emp-inc
 Self-emp-not-inc
?_workclass
 Adm-clerical
 Exec-managerial
 Farming-fishing
 Handlers-cleaners
 Machine-op-inspct
 Other-service
 Priv-house-serv
 Prof-specialty
 Protective-serv
 Sales
 Tech-support
 Transport-moving
?_occupation
 Husband
 Not-in-family
 Other-relative
 Own-child
 Unmarried
 Wife
 Amer-Indian-Eskimo
 Black
 Other
 White
 Columbia
 Dominican-Republic
 El-Salvador
 India
 Mexico
edu_degree
marriage


In [21]:
train_X.head()

,age,fnlwgt,sex,capital_gain,capital_loss,hours_per_week,Federal-gov,Local-gov,Private,Self-emp-inc,...,Black,Other,White,Columbia,Dominican-Republic,El-Salvador,India,Mexico,edu_degree,marriage
0,0.030670,-1.063594,1,0.148451,-0.216656,-0.035429,0,0,0,0,...,0,0,1,0,0,0,0,0,0.272409,-1
1,0.837096,-1.008692,1,-0.145918,-0.216656,-2.222119,0,0,0,0,...,0,0,1,0,0,0,0,0,0.272409,1
2,-0.042641,0.245075,1,-0.145918,-0.216656,-0.035429,0,0,1,0,...,0,0,1,0,0,0,0,0,-0.601905,0
3,1.057031,0.425795,1,-0.145918,-0.216656,-0.035429,0,0,1,0,...,1,0,0,0,0,0,0,0,-1.476218,1
4,-0.775756,1.408154,0,-0.145918,-0.216656,-0.035429,0,0,1,0,...,1,0,0,0,0,0,0,0,0.272409,1


In [22]:
#feature balencing
# new_trainX = train_X.values.tolist()
# new_trainY = train_Y.values.tolist()

# for i in range(len(train_X)):
#     if train_Y.values[i] ==1 and np.random.random()<=0.2:
#         new_trainX.append(new_trainX[i])
#         new_trainY.append(new_trainY[i])
# print(len(new_trainX))
# print(len(new_trainY))
# new_trainX = np.asarray(new_trainX)
# new_trainY = np.asarray(new_trainY).reshape(-1)
# print(new_trainX.shape)
# print(new_trainY.shape)
# print(test_X.shape)

In [29]:
import pickle

In [25]:
clf = SVC(kernel='rbf', random_state=0)
clf.fit(train_X, train_Y)
pickle.dump(clf, open('model.txt', 'wb'))               
# from sklearn.feature_selection import RFE
# svc_lin=SVC(kernel='rbf')
# svm_rfe_model=RFE(estimator=svc_lin)
# svm_rfe_model_fit=svm_rfe_model.fit(train_X, train_Y)
# feat_index = pd.Series(data = svm_rfe_model_fit.ranking_, index = train_X.columns)
# signi_feat_rfe = feat_index[feat_index==1].index
# print('Significant features from RFE',signi_feat_rfe)
# # clf = SVC(kernel='rbf', random_state=0)
# clf.fit(train_X, train_Y)
# pickle.dump(clf, open('model.txt', 'wb'))               


In [26]:
results = clf.predict(test_X)
with open('predict_Howardlinear.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['id','label'])
    for i, x in enumerate(results):
        writer.writerow([i + 1, int(x)])

In [ ]:
olddata=[]
newdata=[]
with open('predict_best.csv', 'r', newline='') as fw:
    with open('predict_Howard01.csv', 'r', newline='') as f:
        csvreader = csv.reader(fw)
        csvreader1 = csv.reader(f)
        for row in csvreader:
            olddata.append(row)
        for row in csvreader1:
            newdata.append(row)
count=0
for i in range(1,len(olddata)):
    if olddata[i][1] != newdata[i][1]:
        count+=1
print(count)

In [31]:
#use model.sav
model = pickle.load(open('model_vector.txt', 'rb'))
results = model.predict(test_X)
with open('predict.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['id','label'])
    for i, x in enumerate(results):
      writer.writerow([i + 1, int(x)])